<a href="https://colab.research.google.com/github/PatrykCiwinski/Spam_detector/blob/main/SpamDetector_transfer_learning_with_Bert_part3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import numpy as np
import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub
import tensorflow_text as text

In [ ]:
!pip install tensorflow-text

In [ ]:
!wget https://lazyprogrammer.me/course_files/spam.csv

In [57]:
df=pd.read_csv('spam.csv', encoding='cp437')

In [58]:
df=df.drop(df.columns[[2,3,4]], axis=1)

In [59]:
df.columns=[['labels','text']]

In [60]:
df.columns = df.columns.get_level_values(0)

In [39]:
df.columns

Index(['labels', 'text'], dtype='object')

In [61]:
df['category'] = df['labels'].map({'ham':0,'spam':1})

In [64]:
df.groupby('labels').size()

labels
ham     4825
spam     747
dtype: int64

In [65]:
#Ther is an imbalance in labels, so we need to cope with it

In [66]:
df_spam = df[df['labels']=='spam']

In [67]:
df_ham = df[df['labels']=='ham']

In [69]:
df_downsample=df_ham.sample(df_spam.shape[0])

In [70]:
new_balanced_df=pd.concat([df_downsample,df_spam])

In [ ]:
new_balanced_df

In [74]:
X_train, X_test, y_train, y_test = train_test_split(new_balanced_df['text'],new_balanced_df['category'] , stratify=new_balanced_df['category'], random_state=42)

In [75]:
bert_preprocess = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')
bert_encoder = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4')

In [76]:
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)
# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)
# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=2, batch_size = 32)

Epoch 1/2
35/35 [==============================] - 22s 345ms/step - loss: 0.6751 - accuracy: 0.5688
Epoch 2/2
35/35 [==============================] - 12s 351ms/step - loss: 0.5428 - accuracy: 0.7884


In [77]:
y_predict = model.predict(X_test)
y_predict = y_predict.flatten()
y_predicted = np.where(y_predict > 0.5, 1,0)

12/12 [==============================] - 5s 363ms/step


In [78]:
y_predicted

array([0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0,
       0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1,
       1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1,
       1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1,
       0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0,
       0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0,
       0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1,
       1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1,
       0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1,

In [79]:
print(classification_report(y_test,y_predicted))

              precision    recall  f1-score   support

           0       0.85      0.92      0.88       187
           1       0.91      0.83      0.87       187

    accuracy                           0.88       374
   macro avg       0.88      0.88      0.88       374
weighted avg       0.88      0.88      0.88       374



In [82]:
def text_predict(text):
  y_predict = model.predict([text])
  y_predict = y_predict.flatten()
  y_predicted = np.where(y_predict > 0.5, 1, 0)
  if y_predicted == 1:
    print('spam')
  else:
    print('not spam')
  return y_predicted

In [87]:
text_predict('You can win a lot of money, register in the link below')

1/1 [==============================] - 0s 44ms/step
spam


array([1])